<a href="https://colab.research.google.com/github/indranildchandra/ML101-Codelabs/blob/master/src/Transfer_Learning_with_Keras_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The building of a model is a 3 step process:

Importing the pre-trained model and adding the dense layers.
Loading train data into ImageDataGenerators.
Training and Evaluating model.

First load the dependencies.

In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


Then import the pre-trained MobileNet model. The Mobilenet (trained on the imagenet dataset for a thousand classes) will have a last layer consisting of 1000 neurons (one for each class). We want as many neurons in the last layer of the network as the number of classes we wish to identify. So we discard the 1000 neuron layer and add our own last layer for the network.

This can be done by setting (IncludeTop=False) when importing the model.

So suppose you want to train a flower classifier to identify 5 different categories, we need 5 neurons in the final layer. This can be done using the following code.

This is step 1 of the process. Importing and building the required model.

We import the MobileNet model without its last layer and add a few dense layers so that our model can learn more complex functions. The dense layers must have the relu activation function and the last layer,which contains as many neurons as the number of classes must have the softmax activation.



In [2]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(5,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
W0616 01:58:37.885844 140592244209536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0616 01:58:37.905111 140592244209536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0616 01:58:37.910338 140592244209536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.un

Next we make a model based on the architecture we have provided.

In [0]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

To check the architecture of our model, we simply need to use this line of code given below.

In [4]:
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_1
1 conv1_pad
2 conv1
3 conv1_bn
4 conv1_relu
5 conv_dw_1
6 conv_dw_1_bn
7 conv_dw_1_relu
8 conv_pw_1
9 conv_pw_1_bn
10 conv_pw_1_relu
11 conv_pad_2
12 conv_dw_2
13 conv_dw_2_bn
14 conv_dw_2_relu
15 conv_pw_2
16 conv_pw_2_bn
17 conv_pw_2_relu
18 conv_dw_3
19 conv_dw_3_bn
20 conv_dw_3_relu
21 conv_pw_3
22 conv_pw_3_bn
23 conv_pw_3_relu
24 conv_pad_4
25 conv_dw_4
26 conv_dw_4_bn
27 conv_dw_4_relu
28 conv_pw_4
29 conv_pw_4_bn
30 conv_pw_4_relu
31 conv_dw_5
32 conv_dw_5_bn
33 conv_dw_5_relu
34 conv_pw_5
35 conv_pw_5_bn
36 conv_pw_5_relu
37 conv_pad_6
38 conv_dw_6
39 conv_dw_6_bn
40 conv_dw_6_relu
41 conv_pw_6
42 conv_pw_6_bn
43 conv_pw_6_relu
44 conv_dw_7
45 conv_dw_7_bn
46 conv_dw_7_relu
47 conv_pw_7
48 conv_pw_7_bn
49 conv_pw_7_relu
50 conv_dw_8
51 conv_dw_8_bn
52 conv_dw_8_relu
53 conv_pw_8
54 conv_pw_8_bn
55 conv_pw_8_relu
56 conv_dw_9
57 conv_dw_9_bn
58 conv_dw_9_relu
59 conv_pw_9
60 conv_pw_9_bn
61 conv_pw_9_relu
62 conv_dw_10
63 conv_dw_10_bn
64 conv_dw_10_relu
65 conv_pw_10

Now that we have our model, as we will be using the pre-trained weights, that our model has been trained on (imagenet dataset), we have to set all the weights to be non-trainable. We will only be training the last Dense layers that we have made previously. The code for doing this is given below.

In [0]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

Now we will download the flower photos dataset from tensorflow's repo.

In [0]:
!ls -ltr

In [8]:
%%time

from six.moves import urllib

FLOWERS_DIR = './flower_photos'

if not os.path.exists(FLOWERS_DIR):
  DOWNLOAD_URL = 'http://download.tensorflow.org/example_images/flower_photos.tgz'
  print('Downloading flower images from %s...' % DOWNLOAD_URL)
  urllib.request.urlretrieve(DOWNLOAD_URL, 'flower_photos.tgz')
  !tar xfz flower_photos.tgz
print('Flower photos are located in %s' % FLOWERS_DIR)

Flower photos are located in ./flower_photos


In [7]:
!ls -ltr

total 223460
drwxr-x--- 7 270850 5000      4096 Feb 10  2016 flower_photos
drwxr-xr-x 1 root   root      4096 May 31 16:17 sample_data
-rw-r--r-- 1 root   root 228813984 Jun 16 01:46 flower_photos.tgz


Now we move onto Step 2 of the process, loading the training data into the ImageDataGenerator.

ImageDataGenerators are inbuilt in keras and help us to train our model. We just have to specify the path to our training data and it automatically sends the data for training, in batches. It makes the code much simpler.

For that we need our training data in a particular format as mentioned earlier in the blog.

In [8]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('./flower_photos/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 3670 images belonging to 5 classes.


Next we move onto Step 3, training the model on the dataset.

For this we first compile the model that we made, and then train our model with our generator. This can be done using the code below.

In [9]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5)

W0616 01:59:25.011923 140592244209536 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0616 01:59:25.137627 140592244209536 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
114/114 [==============================] - 19s 167ms/step - loss: 0.5616 - acc: 0.8163
Epoch 2/5
114/114 [==============================] - 14s 126ms/step - loss: 0.3598 - acc: 0.8824
Epoch 3/5
114/114 [==============================] - 14s 124ms/step - loss: 0.2826 - acc: 0.9123
Epoch 4/5
114/114 [==============================] - 14s 123ms/step - loss: 0.2486 - acc: 0.9295
Epoch 5/5
114/114 [==============================] - 14s 123ms/step - loss: 0.2099 - acc: 0.9341
